In [2]:
import os
from math import ceil
from dotenv import load_dotenv

load_dotenv("keys.env")
import polars as pl

from polymarket_apis.clients.clob_client import PolymarketClobClient
from polymarket_apis.types.gamma_types import Event
# from poly_trader.helpers import get_all_order_books_async, get_tops_by_event, simplify_event

In [3]:
clob = PolymarketClobClient(private_key=os.getenv("PRIVATE_KEY"), proxy_address=os.getenv("PROXY_ADDRESS"))

In [4]:
neg_risk_events_df = pl.read_parquet("data/neg_risk_events.parquet")
events_df = neg_risk_events_df.filter(pl.col("negRiskAugmented") == False)
events = [Event(**event) for event in events_df.to_dicts()]

AttributeError: 'datetime.datetime' object has no attribute 'endswith'

In [5]:
a = Event(**neg_risk_events_df.to_dicts()[0])

AttributeError: 'datetime.datetime' object has no attribute 'endswith'

In [25]:
simple_events = [simplify_event(event) for event in events]
yes_tokens = [market.token_ids[0] for event in simple_events for market in event.markets]
no_tokens = [market.token_ids[1] for event in simple_events for market in event.markets]
yes_books = await get_all_order_books_async(clob=clob, token_ids=yes_tokens, batch_size=ceil(len(yes_tokens) / 3))
yes_books_dict = {book.token_id: book for book in yes_books}
tops_by_event = get_tops_by_event(simple_events, yes_books_dict)

In [26]:
    # Calculate sums and find smallest sizes for each event
result_list = []
for event_slug, event_data in tops_by_event.items():
    sum_bids = round(sum(market['top']['bid']['price'] for market in event_data['tops']),3)
    sum_asks = round(sum(market['top']['ask']['price'] for market in event_data['tops']),3)

    if 0.1 < sum_asks < 1:
        bid_sizes = [market['top']['bid']['size'] for market in event_data['tops']]
        ask_sizes = [market['top']['ask']['size'] for market in event_data['tops']]

        smallest_bid_size = min(bid_sizes) if bid_sizes else 0
        smallest_ask_size = min(ask_sizes) if ask_sizes else 0

        result_list.append([
            sum_bids,
            smallest_bid_size,
            sum_asks,
            smallest_ask_size,
            event_slug,
        ])

for result in result_list:
    print(result)

[0.964, 6.36, 0.988, 7.39, 'will-bitcoin-activate-op-ctv-or-op-cat-in-2025']


In [27]:
slugs = [item[4] for item in result_list]
yes_arbs_list_df = neg_risk_events_df.filter(pl.col("slug").is_in(slugs))
yes_arbs_list_df.write_parquet("data/yes_arbs_list.parquet")

In [28]:
# token_id_to_find = "6427519506286419532134012763551211259629796084507143468688888941057296525505"
#
# matching_event = next(
#     (event for event in simple_events if any(market.token_ids and market.token_ids[0] == token_id_to_find for market in event.markets)),
#     None
# )
#
# if matching_event:
#     print(f"Found event: {matching_event}")
# else:
#     print("No matching event found.")